In [1]:
!pip3 install pwntools

     --------------------------------------- 11.7/11.7 MB 11.7 MB/s eta 0:00:00
     ---------------------------------------- 90.6/90.6 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 1.3/1.3 MB 11.6 MB/s eta 0:00:00
     ---------------------------------------- 78.6/78.6 kB ? eta 0:00:00
     --------------------------------------- 15.7/15.7 MB 11.5 MB/s eta 0:00:00
     ------------------------------------- 177.6/177.6 kB 10.5 MB/s eta 0:00:00
     ---------------------------------------- 74.0/74.0 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ------------------------------------- 224.8/224.8 kB 13.4 MB/s eta 0:00:00
     ------------------------------------- 212.1/212.1 kB 12.6 MB/s eta 0:00:00
     ---------------------------------------- 2.7/2.7 MB 10.6 MB/s eta 0:00:00
     

In [15]:
!pip3 install pycryptodome

     ---------------------------------------- 1.7/1.7 MB 9.2 MB/s eta 0:00:00


In [1]:
#!/usr/bin/env python3
import os
import sys
from Crypto.Cipher import AES
from Crypto.Util import Counter
from binascii import hexlify

def log(*err_messages):
	'''function for debugging purposes'''
	logs = open('err.log','a')
	for msg in err_messages:
		if type(msg) == bytes: msg = hexlify(msg).decode()
		logs.write(msg)
	logs.write('\n')
	logs.close()

def enc(msg : bytes, key : bytes) -> bytes:
	'''Encrypt message with AES'''
	aes = AES.new(key, AES.MODE_CTR, counter = Counter.new(128,initial_value = 0))
	return aes.encrypt(msg)

dec = enc #corresponding decryption function

def encrypt(msg, key : bytes):
	'''Encrypt message with ONe-Time-Pad'''
	if type(msg) == str: msg = msg.encode()
	return hexlify(bytes([msg[i] ^ key[i % 32] for i in range(len(msg))])).decode()

if __name__ == '__main__':
	try:
		flag = open('flag','r').read().strip()
		key = os.urandom(32)
		cipher = enc(flag.encode(), key)
		print(hexlify(cipher).decode())
		log('cipher: ', cipher)
		log('key: ', key)
		print('I will encrypt a single message of length at most 30 for you.\nMessage: ')
		sys.stdout.flush()
		message = sys.stdin.buffer.readline().strip()
		assert len(message) <= 30
		print(encrypt(message, key))
	except Exception as err:
		print('Something went wrong')
		log('ERROR: ', repr(err))


Something went wrong


In [26]:
from binascii import unhexlify

def decodeOTP(plaintext, cipher): # plaintext in string, cipher as hex
    cipher = unhexlify(cipher)
    plaintext = plaintext.encode()
    # key = bytes([plaintext[i] ^ cipher[i] for i in range(len(plaintext))])
    key = unhexlify(encrypt(plaintext, cipher))
    return key

string_crack = "AAAAAAAAAAAAAAAAAAAAAAAAAAAAAA"

def decodeAES(cipher, key): # cipher as hex, key as bytes
    # key is only 30 bytes long, so we need to brute force the last 2 bytes
    for i in range(256):
        for j in range(256):
            try:
                key = key + bytes([i]) + bytes([j])
                aes = AES.new(key, AES.MODE_CTR, counter = Counter.new(128,initial_value = 0))
                plaintext = aes.decrypt(unhexlify(cipher))
                if b"ENO" in plaintext:
                    print(plaintext)
                    print(i, j)
            except:
                pass
            key = key[:-2]

decodeAES("f0a8fdd6183fb8818f497097dbf52aa57a4736ed5a043f73b1bd8ddb74a78bcd", decodeOTP(string_crack, "8aa96e037f2a3706287855213df9713ac307a92de2a1c31df99c1fc4f1a0"))

b'ENO{r3u5e_i5_0n1y_f0r_re4l_l1fe}'
246 10
